In [37]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
import dedupe
import re
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [3]:
# Residential Land Use Codes from MA Dept of Revenue
# https://www.mass.gov/files/documents/2016/08/wr/classificationcodebook.pdf
# Codes are 101*-109*, 031*, and 013*
# Often include suffixes (letters, zeroes or no character), thus regex *?
USE_CODES = '^1[0-1][1-9]*?|^013*?|^031*?'
BOS_CODES = '^R[1-4]$|^RC$|^RL$|^CD$|^A$'
def read_res(file_dict):
    df = pd.DataFrame()
    for town, file, in file_dict.items():
        town_df = gpd.read_file(file).drop('geometry', axis='columns')
        town_df['town'] = town
        df = df.append(town_df, ignore_index=True)
    return df

In [4]:
# Data from MassGIS Standardized Assessor's Parcels
# https://docs.digital.mass.gov/dataset/massgis-data-standardized-assessors-parcels
# Medford, Cambridge, and Somerville all last updated FY 2019

In [5]:
# Medford processing.
files = {'med': './data/assess/med_assess.dbf'}
med = read_res(files)
# Rename column to lower-case.
med.columns = med.columns.str.lower()
# Filter for residential parcels.
med = med[med['use_code'].str.contains(USE_CODES, regex=True)]
# Identify rows with co-owner names erroneously listed in address column.
mask = med.own_addr.str.contains(pat = '|'.join(['^C/O', '^[A-Za-z]']), na=False) & ~med.own_addr.str.contains(pat = '|'.join(['^PO', '^P.O.', '^P. O.', '^P O ', '^ONE', '^BOX', '^ZERO']), na=False)
# Add co-owners identified to co column.
med['co'] = med.own_addr[mask]
med.loc[~mask, 'co'] = None
# Fill own_addr with none for above-identified rows.
med.loc[mask, 'own_addr'] = None
# Remame columns
med = med.rename(columns = {
    'prop_id': 'gisid',
    'owner1': 'own',
    'site_addr': 'prop_addr',
    'total_val': 'ass_val',
    'location': 'unit'
})
med.loc[:,'prop_addr'] = med.prop_addr.str.strip()
# Replace underscores with hyphens.
med.loc[:,'gisid'] = med.gisid.str.replace(r'_', '-', regex=True)
# Concatenate address.
med.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c))) for a, b, c in zip(med['own_addr'], med['own_city'], med['own_state'])]
med.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(med['own_addr'], med['own_zip'])]
# Remove concatenated Nones.
med = med.replace({r'None, ': ''}, regex=True)
med['year'] = 'FY2019'
# Filter columns.
med = med[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year']]

In [81]:
# Somerville processing.
som = pd.read_csv('./data/assess/som_assess_FY14-FY19.csv', dtype={'HOUSE NO': str})
som.columns = som.columns.str.lower()

# Assessor seems to have screwed up this column in the 2014-2019 data
# but it appears that 2019 data is incrementally numbered (¯\_(ツ)_/¯)
som = som.loc[som['fiscal_year'] >= 2019]

# Filter for residential parcels.
som = som[som['pcc'].str.contains(USE_CODES, regex=True)]

som.loc[:,'prop_addr'] = [' '.join((str(a), str(b))) for a, b in zip(som['house no'], som['street'])]

som = som.replace({r'^, ': '', r' ,': '', r', nan': '', r'nan': '', r'None, ': '', r', None': ''}, regex=True)
som = som.replace({' ': None, '': None, np.nan: None})

# Pad ZIP code with zeroes, remove 4-digit suffix.
som.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c), str(d))) for a, b, c, d in zip(som['owner add'], som['owner city'], som['owner state'], som['owner zip'])]

som.loc[:,'gisid']= ['-'.join((str(m), str(b), str(l))) for m, b, l in zip(som['map'], som['block'], som['lot'])]

som['fiscal_year'] = 'FY2019'
som['town'] = 'som'
som = som.drop(['year'], axis=1)
som = som.rename(columns = {
    'commitment owner': 'own',
    'current co-owner': 'co',
    'parcel val': 'ass_val',
    'fiscal_year': 'year'
})
# Filter columns.
som = som[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year']]

/home/ericrobskyhuntley/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (13,20,53,54,59,60,61,62,63,64,65,66,67,68,69,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [82]:
bos = pd.read_csv('./data/assess/bos_assess.csv', dtype={'GIS_ID': str, 'MAIL_ZIPCODE': str, 'U_TOT_RMS': str})
bos.columns = bos.columns.str.lower()
bos = bos.rename(columns = {
    'gis_id': 'gisid',
    'owner': 'own',
    'mail_addressee': 'co',
    'unit_num': 'unit',
    'av_total': 'ass_val'
})
bos['town'] = 'bos'
# Filter by residential property types.
bos = bos[bos['lu'].str.contains(BOS_CODES, regex=True)]
bos.loc[:, 'gisid'] = bos.gisid.str.strip().str.pad(width=10, side='left', fillchar='0')
# Pad ZIP code with zeroes, remove 4-digit suffix.
bos.loc[:,'mail_zipcode'] = bos.mail_zipcode.astype(str).str.strip().str.pad(width=5, side='left', fillchar='0')
# Add comma between city and state.
bos.loc[:,'mail cs'] = bos['mail cs'].str.rsplit(' ', 1).apply(lambda x: ', '.join(x))
# Concatenate property address components
bos.loc[:,'prop_addr'] = [' '.join((str(a), str(b), str(c))) for a, b, c in zip(bos['st_num'], bos['st_name'], bos['st_name_suf'])]
bos.loc[:,'prop_addr'] = bos.prop_addr.str.strip()
# Concatenate owner address components.
bos.loc[:,'own_addr'] = [', '.join((str(a), str(b))) for a, b in zip(bos['mail_address'], bos['mail cs'])]
bos.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(bos['own_addr'], bos['mail_zipcode'])]
bos.loc[:,'own_addr'] = bos.own_addr.str.strip()
# Filter columns
bos['year'] = 'FY2020'
bos = bos[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year']]
# Replace blank strings with None (necessary for dedupe).
bos = bos.replace({' ': None, '': None, r' #nan': None})
bos = bos.replace({r' #nan': ''}, regex=True)

In [83]:
cam = pd.read_csv('./data/assess/cam_assess.csv', dtype=str)
# rename all columns to lowercase
cam.columns = cam.columns.str.lower()
# Filter for residential properties.
cam = cam[cam['stateclasscode'].str.contains(USE_CODES, regex=True)]
# Pad zip to five digits and remove 4-digit zip suffix.
cam.loc[:,'owner_zip'] = cam['owner_zip'].str.rsplit('-', 1).str[0]
# Identify rows with co-owner names erroneously listed in address column.
mask = cam.owner_address.str.contains(pat = '|'.join(['^C/O', '^ATTN:']), na=False)
cam.loc[mask, 'owner_address'] = None
# Add co-owners identified to co column.
cam.loc[mask, 'owner_coownername'] = [', '.join((str(a), str(b)))  for a, b in zip(cam.loc[mask, 'owner_coownername'], cam.loc[mask, 'owner_address'])]
# Concatenate owner address components
cam.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c), str(d))) for a, b, c, d in zip(cam['owner_address'], cam['owner_address2'], cam['owner_city'], cam['owner_state'])]
cam.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(cam['own_addr'], cam['owner_zip'])]
cam.loc[:,'own_addr'] = cam.own_addr.str.strip()
# Clean property address column
cam['prop_addr'] = cam['address'].str.rsplit('\nCambridge, MA', 1).apply(lambda x: x[0].replace('\n', ' ').strip())
# Bring property address in line with others.
cam['town'] = 'cam' 
cam = cam.rename(columns = {
    'owner_name': 'own',
    'owner_coownername': 'co',
    'assessedvalue': 'ass_val'
})
cam['year'] = 'FY2020'
cam = cam.replace({r'^, ': '', r' ,': '', r', nan': '', r'None, ': '', r', None': ''}, regex=True)
cam = cam.replace({' ': None, '': None, np.nan: None})
cam = cam[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year']]

In [84]:
brook = pd.read_csv('./data/assess/brook_assess.csv', dtype=str)
brook.columns = brook.columns.str.lower()
brook = brook[brook['usecd'].str.contains(USE_CODES, regex=True)]

brook.loc[:,'zip'] = brook['zip'].str.rsplit('-', 1).str[0]
# Name town.
brook['town'] = 'brk' 
# Concatenate address.
brook.loc[:,'own_addr'] = [', '.join((str(a), str(b), str(c))) for a, b, c, in zip(brook['address'], brook['city'], brook['state'])]
# Append zip to address with no comma.
brook.loc[:,'own_addr'] = [' '.join((str(a), str(b))) for a, b in zip(brook['own_addr'], brook['zip'])]
brook.loc[:,'own_addr'] = brook.own_addr.str.strip()
# Concatenate property address components
brook.loc[:,'prop_addr'] = [''.join((str(a), str(b))) for a, b in zip(brook['addno1'], brook['addno2'])]
brook.loc[:,'prop_addr'] = [' '.join((str(a), str(b))) for a, b in zip(brook['prop_addr'], brook['addst1'])]
brook.loc[:,'prop_addr'] = brook.prop_addr.str.strip()
# Append 
brook.loc[:,'own'] = [' '.join((str(a), str(b))) for a, b in zip(brook['firstname1'], brook['lastname1'])]
brook.loc[:,'co'] = [' '.join((str(a), str(b))) for a, b in zip(brook['firstname2'], brook['lastname2'])]
brook = brook.replace({' ': None, '': None})
brook = brook.rename(columns = {
    'parcel-id': 'gisid',
    'addst2': 'unit',
    'restotlval': 'ass_val'
})
brook = brook.replace({r'^, ': '', r' ,': '', r', nan': '', r'nan': '', r'None, ': '', r', None': ''}, regex=True)
brook = brook.replace({' ': None, '': None})
brook['year'] = 'FY2020'
brook = brook[['gisid', 'town', 'prop_addr', 'unit', 'own', 'co', 'own_addr', 'ass_val', 'year']]

In [95]:
all_assess = pd.concat([som, med, cam, bos, brook], ignore_index=True)

In [96]:
all_assess.loc[:,'prop_addr'] = all_assess.prop_addr.str.lstrip('0').str.strip()
all_assess.loc[:,'own_addr'] = all_assess.own_addr.str.lstrip('0').str.strip()
all_assess.loc[:,'co'] = all_assess['co'].replace({r'C/O ': '', r'S/O ': '', r'ATTN: ': '', r'ATTN ': ''}, regex=True)
all_assess = all_assess.replace({r'None': '', 'nan': ''}, regex=True)
all_assess = all_assess.replace({' ': None, '': None})
all_assess = all_assess[~pd.isnull(all_assess['gisid'])]

In [97]:
# Convert to dictionary (expected by Dedupe)
all_assess_dict = all_assess.to_dict('index')

In [98]:
settings_file = './training/learned_settings'
training_file = './training/training.json'

# If settings exist, read from existing.
if os.path.exists(settings_file):
    print('Reading learned settings from', settings_file)
    with open(settings_file, 'rb') as f:
        deduper = dedupe.StaticDedupe(f)
else:
    # Tell Dedupe which fields are used to identify duplicates.
    fields = [
        {'field': 'own', 'variable name': 'own', 'type': 'Name'},
        {'field': 'co', 'variable name': 'co', 'type': 'Name'},
        {'field': 'own_addr', 'variable name': 'own_addr', 'type': 'Address'},
        {'type': 'Interaction', 'interaction variables': ['own', 'co']}
        ]
    deduper = dedupe.Dedupe(fields)
    # If training file exists, read it...
    if os.path.exists(training_file):
        print('reading labeled examples from ', training_file)
        with open(training_file, 'rb') as f:
            deduper.prepare_training(all_assess_dict, f)
    # Otherwise, prepare a training set...
    else:
        deduper.prepare_training(all_assess_dict)
    # Start supervised labeling.
    dedupe.console_label(deduper)
    deduper.train()
    # Write settings and training sets.
    with open(training_file, 'w') as tf:
        deduper.write_training(tf)
    with open(settings_file, 'wb') as sf:
        deduper.write_settings(sf)

INFO:dedupe.api:((SimplePredicate: (oneGramFingerprint, own_addr), SimplePredicate: (sameThreeCharStartPredicate, own_addr)), (PartialPredicate: (commonTwoTokens, own, CorporationName), PartialPredicate: (twoGramFingerprint, own, CorporationName)), (PartialPredicate: (commonTwoTokens, co, CorporationName), SimplePredicate: (commonThreeTokens, own_addr)), (PartialPredicate: (commonFourGram, own, Surname), SimplePredicate: (twoGramFingerprint, own)))


Reading learned settings from ./training/learned_settings


In [101]:
# Identify clusters based on training dataset.
# Higher threshold is less tolerant of differences between names/addresses.
clustered_dupes = deduper.partition(all_assess_dict, threshold = 0.7)

# How many sets are there?
print('Number of sets', len(clustered_dupes))

INFO:dedupe.blocking:10000, 7.0981082 seconds
INFO:dedupe.blocking:20000, 12.3324822 seconds
INFO:dedupe.blocking:30000, 16.8092832 seconds
INFO:dedupe.blocking:40000, 22.9086502 seconds
INFO:dedupe.blocking:50000, 29.4327222 seconds
INFO:dedupe.blocking:60000, 34.7913572 seconds
INFO:dedupe.blocking:70000, 39.0066132 seconds
INFO:dedupe.blocking:80000, 43.5724902 seconds
INFO:dedupe.blocking:90000, 48.1304922 seconds
INFO:dedupe.blocking:100000, 52.6081892 seconds
INFO:dedupe.blocking:110000, 57.3003302 seconds
INFO:dedupe.blocking:120000, 61.6256222 seconds
INFO:dedupe.blocking:130000, 65.6209532 seconds
INFO:dedupe.blocking:140000, 69.7389402 seconds
INFO:dedupe.blocking:150000, 74.2687262 seconds
INFO:dedupe.blocking:160000, 78.6503052 seconds
INFO:dedupe.blocking:170000, 82.8063082 seconds
INFO:dedupe.blocking:180000, 87.5153912 seconds
INFO:dedupe.blocking:190000, 91.3037522 seconds
INFO:dedupe.blocking:200000, 95.8212092 seconds
INFO:dedupe.blocking:210000, 100.8320282 seconds


Number of sets 175344


In [102]:
# Create empty arrays to hold results.
rid = []
clst = []
conf = []
count = []

# Iterate over results...
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        # Append record id
        # Corresponds to index of assessor dataframe.
        rid.append(record_id)
        # Append cluster ID.
        clst.append(cluster_id)
        # Append confidence score.
        conf.append(score)

# Build new dataframe using result arrays.
clust = pd.DataFrame(list(zip(clst, conf)), 
                  columns =['clst', 'conf'],
                  index = rid
                 )

In [103]:
# Join clusters to assessors dataframe.
all_assess = all_assess.join(clust)

In [104]:
# Read spatial data
parcels_gdf = gpd.read_file('./data/parcels/mamas_parcels.shp')
parcels_gdf = parcels_gdf.rename(columns = {
    'pid': 'gisid'
}).drop_duplicates(subset=['gisid', 'town'])
parcels_gdf = parcels_gdf[~pd.isnull(parcels_gdf['gisid'])]
parcels_gdf = parcels_gdf[~pd.isnull(parcels_gdf['geometry'])]
# parcels_gdf.loc[:,'geometry'] = parcels_gdf.geometry.centroid
centroid = parcels_gdf.geometry.centroid
parcels_gdf.loc[:,'lat'] = centroid.y
parcels_gdf.loc[:,'lon'] = centroid.x

<ipython-input-104-7d7fbb14c5ae>:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = parcels_gdf.geometry.centroid


In [17]:
def prop_count(group):
    cnt = group['gisid'].count()
    group['count'] = cnt
    return group

In [105]:
all_assess = parcels_gdf.merge(all_assess, on=['town', 'gisid'], how='right')
all_assess = all_assess[~np.isnan(all_assess.lat)]

In [106]:
# Hard-coding this count logic saves a ton of time for each PostgreSQL query.
all_assess = all_assess.merge(all_assess.groupby('clst').count()[['gisid']].rename(columns={'gisid': 'count'}), on=['clst', 'clst'], how='left')

In [79]:
all_assess

,gisid,town,geometry,lat,lon,prop_addr,unit,own,co,own_addr,ass_val,year,clst,conf,count
0,2-A-25,som,"POLYGON ((-71.12989 42.41378, -71.13021 42.413...",42.413673,-71.130108,75 GORDON ST,None,KARGATZIS-FEMINO KIRIAKOULA CYNDIE TR,KOSTAS KARGATZIS 1997 FAMILY TRUST,"279 PORTER ST, MELROSE, MA, 002176",805000,2019,24504,1.000000,1
1,2-A-24,som,"POLYGON ((-71.12977 42.41372, -71.13009 42.413...",42.413606,-71.129988,73 GORDON ST,None,DIFUSCO FRANCO & FILOMENA TRUSTEES,F & F DIFUSCO IRREVOCABLE TRUST,"73 GORDON ST, SOMERVILLE, MA, 002144",857200,2019,22432,1.000000,1
2,3-C-2,som,"POLYGON ((-71.13051 42.41354, -71.13069 42.413...",42.413493,-71.130657,90 GORDON ST,None,SACHER MATHEW J,None,"103 WESTGATE RD, WELLESLEY, MA, 002481",795100,2019,26284,1.000000,1
3,3-C-3,som,"POLYGON ((-71.13039 42.41347, -71.13058 42.413...",42.413423,-71.130543,86 GORDON ST,86,POMMET DAVID MICHAEL,None,"86 GORDON ST, SOMERVILLE, MA, 002144",362800,2019,30593,1.000000,1
4,3-C-3,som,"POLYGON ((-71.13039 42.41347, -71.13058 42.413...",42.413423,-71.130543,88 GORDON ST,88,POMMET TONI MARIE,None,"88 GORDON ST, SOMERVILLE, MA, 002144",522200,2019,30594,1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205444,2205664000,bos,"POLYGON ((-71.16615 42.34100, -71.16616 42.340...",42.340946,-71.166342,18 16 LAKE ST,NaN,HOFFMAN ANN MARIE,None,"18 LAKE ST, BRIGHTON, MA 02135",968700,FY2020,161679,1.000000,1
205445,2205665000,bos,"POLYGON ((-71.16616 42.34086, -71.16617 42.340...",42.340797,-71.166342,14 LAKE ST,2,LIU PI-YAO A,None,"14 LAKE ST #2, BRIGHTON, MA 02135",479400,FY2020,161680,1.000000,1
205446,2205665000,bos,"POLYGON ((-71.16616 42.34086, -71.16617 42.340...",42.340797,-71.166342,12 LAKE ST,1,LIU SHIK-CHUN DAVID TS,C/O SHIH CHUN,"110 ALGONQUIN RD, NEWTON, MA 02467",462200,FY2020,16929,0.796217,5
205447,2205667000,bos,"POLYGON ((-71.16673 42.34040, -71.16676 42.340...",42.340527,-71.166669,LAKE ST,NaN,GREALISH MARTIN J TS,None,"111 HUNTINGTON AV 12TH FLR, BOSTON, MA 02199",58300,FY2020,161681,1.000000,1


In [107]:
pg_engine = create_engine(PG_CONNECT)
all_assess.to_postgis("props", con=pg_engine, schema='public', if_exists='replace', index=True, index_label='id')